In [23]:
import requests 
import bs4 as bs 
from time import sleep
from time import time
from random import randint
import pandas as pd 

movie_title = []
movie_date = []
movie_country = []
movie_genre = []
movie_rate = []

re = requests.get('https://www.imdb.com/feature/genre/?ref_=nv_ch_gr')
soup= bs.BeautifulSoup(re.content, 'html.parser')

In [24]:
genres=[]
containers=soup.find_all('div',class_='table-cell primary')
for i in range(24):    
    genres.append(containers[i].text.strip('  '))
print(genres)


['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Superhero', 'Thriller', 'War', 'Western']


In [25]:
for genre in genres:
    for page in range(1000):
        if page == 0:#先访问第一个列表页面
            cinema = requests.get('https://www.imdb.com/search/title?title_type=feature&genres='+genre+'&sort=user_rating,asc&explore=genres')
        else:#随后访问其他页面
            page = page*50+1
            page = str(page)
            cinema = requests.get('https://www.imdb.com/search/title?title_type=feature&genres='+genre+'&sort=user_rating,asc&start='+page+'&explore=genres&ref_=adv_nxt')
        cinema_soup = bs.BeautifulSoup(cinema.content,'html.parser')
        cinema_containers = cinema_soup.find_all('h3',class_="lister-item-header")
        i=0  #电影页面计数器初始化
        
        for cinema_container in cinema_containers:
            a_cinema_container = cinema_container.find('a',href = True)  #寻找列表中的电影页面的url
            url = a_cinema_container.get('href')
            movie = requests.get('https://www.imdb.com'+ url )
            movie_soup = bs.BeautifulSoup(movie.content,'html.parser')  #打开列表中的电影页面
            movie_rate_single = movie_soup.find('span',itemprop="ratingValue")  #寻找电影评分
            movie_rate_single = float(movie_rate_single.text)
            i = i + 1  #打开一个页面自增一次
            
            if movie_rate_single <= 4:#打开评分小于等于4的电影时
                print(i)
                print(a_cinema_container.text.strip())
                print(movie_rate_single)
                movie_year = cinema_container.find('span',class_="lister-item-year text-muted unbold")   #寻找发行年份并整理
                movie_year = movie_year.text
                movie_year = movie_year[-6:]
                movie_year = movie_year.rstrip(')')
                movie_year = movie_year.lstrip('(')
                movie_year = int(movie_year)
                print(movie_year)
                print('\n')
                if movie_year <= 2017 and movie_year >= 2013:  #寻找时间合适的电影
                    movie_subtext = movie_soup.find('div',class_="subtext")  #寻找genre和date
                    movie_subtext = movie_subtext.find_all('a',href = True)
                    movie_date_single = movie_subtext[-1]  #将最后一个赋值给date
                    movie_genre_single = movie_subtext[:-2]  #将最后一个之前的赋值给genre
                    for m in range(len(movie_genre_single)):  #以text格式整理genre元素
                        movie_genre_single[m] = movie_genre_single[m].text.strip()
                    movie_country_single = movie_soup.find('div',class_="article",id="titleDetails")  #寻找text‘country’
                    movie_country_single = movie_country_single.find('h4',class_="inline",text="Country:")
                    if movie_country_single !=None:
                        movie_country_single = movie_country_single.find_parent()  #寻找text‘country’的父节点
                        movie_country_single = movie_country_single.find('a').text.strip()  #在此节点中寻找country
                    print(movie_country_single)
                      #将元素或列表插入列表
                    movie_title.append(a_cinema_container.text.strip())
                    movie_date.append(movie_date_single.text.strip())
                    movie_country.append(movie_country_single)
                    movie_genre.append(movie_genre_single)
                    movie_rate.append(movie_rate_single)
            else:  #当在一个列表中打开评分大于4的电影时
                break  #跳出当前循环
                
        if movie_rate_single> 4:
            break        
            
IMDBmovie = {
    'title': movie_title,
    'date': movie_date,
    'country': movie_country,
    'genre': movie_genre,
    'rate': movie_rate,
}
pd_IMDB = pd.DataFrame.from_dict(IMDBmovie, orient ='index').transpose()
pd_IMDB.to_csv('pd_IMDB.csv')        
        
    

1
Terror Trail
1.0
1921


2
Boomerang Bill
1.0
1922


3
The Abandoned Project X
1.0
2013


USA
4
Ushikoku ni mairu
1.0
2015


Japan
5
Mstitel chelovechestva
1.0
2016


Russia
6
Raul Roldan
1.0
1952


7
Agent Beetle
1.1
2012


8
Half Way to Hell
1.2
1961


9
Taxi mortal
1.2
1980


10
Frankenpimp
1.2
2009


11
Avenging Force: The Scarab
1.2
2010


12
Pirates: Quest for Snake Island
1.2
2009


13
Mantikora
1.2
2011


14
Thunderstorm: The Return of Thor
1.2
2011


15
Rise of the Black Bat
1.2
2012


16
Browncoats: Independence War
1.2
2015


USA
17
The Living Dead
1.2
2017


Russia
18
Razing Sera
1.2
2017


USA
19
Leather and Iron
1.3
2002


20
Iron Soldier
1.3
2010


21
Acid Head: The Buzzard Nuts County Slaughter
1.3
2011


22
Futures Past
1.3
2012


23
Captain Battle: Legacy War
1.3
2013


USA
24
Ninja the Monster
1.3
2015


Japan
25
The Flying U Ranch
1.4
1927


26
Tsarsulis sashinelebani 2 (atasis pasad)
1.4
1925


27
Shirley Pimple in the John Wayne Temple of Doom
1.4
2000


28
Macar

48
King of the Lost World
2.3
2005


49
Circadian Rhythm
2.3
2005


50
Okka Magadu
2.3
2008


1
Enemies Among Us
2.3
2010


2
Ninja Zombies
2.3
2011


3
Destiny
2.3
2007


4
Lost Woods
2.3
2012


5
Zombie Massacre
2.3
2013


Italy
6
Big Bad Bugs
2.3
2012


7
Solid State
2.3
2012


8
Sins Expiation
2.3
2012


9
Sharlatan
2.3
2005


10
Richard the Lionheart
2.3
2013


USA
11
Sabse Badhkar Hum
2.3
2002


12
Tian ji: Fu chun shan ju tu
2.3
2013


China
13
Starship: Rising
2.3
2014


USA
14
Alex Pandian
2.3
2013


India
15
Avalanche Sharks
2.3
2014


Canada
16
Deportees 3
2.3
2011


17
Chasovshchik
2.3
2013


Russia
18
Martian Land
2.3
2015


USA
19
Sharkenstein
2.3
2016


USA
20
Astro
2.3
2018


21
Complitation of Horror
2.3
2017


Russia
22
Alien Siege
2.3
2018


23
Nazi Overlord
2.3
2018


24
Battle Bots
2.3
2018


25
The Silent Flyer
2.4
1926


26
Mitchell
2.4
1975


27
Shark: Rosso nell'oceano
2.4
1984


28
Warrior of the Lost World
2.4
1983


29
American Commandos
2.4
1986


30
Dragon

9
Realms Hunt for the Shadow Man
2.7
2016


USA
10
In the Name of Ben Hur
2.7
2016


USA
11
Machine
2.7
2017


India
12
Pakka
2.7
2018


13
The Krays: Dead Man Walking
2.7
2018


14
Stark Mad
2.8
1929


15
Man Against Woman
2.8
1932


16
The Cyclone Kid
2.8
1942


17
Ski Troop Attack
2.8
1960


18
Angels' Wild Women
2.8
1972


19
Viva Knievel!
2.8
1977


20
Tie dan ying xiong
2.8
1978


21
Up from the Depths
2.8
1979


22
Search and Destroy
2.8
1988


23
Alienator
2.8
1990


24
Cold Heat
2.8
1989


25
Mob War
2.8
1989


26
American Ninja 5
2.8
1993


27
Point of Impact
2.8
1993


28
Galaxis
2.8
1995


29
Virus
2.8
1996


30
Timelock
2.8
1996


31
Wirus
2.8
1996


32
3 Ninjas: High Noon at Mega Mountain
2.8
1998


33
Steel
2.8
1997


34
Termination Man
2.8
1998


35
Karzan, il favoloso uomo della jungla
2.8
1972


36
Loyal Opposition
2.8
1998


37
Wings of the Crow
2.8
2000


38
Shiberia Chôtokkyû
2.8
1996


39
Mach 2
2.8
2000


40
Hum To Mohabbat Karega
2.8
2000


41
Gangland
2.8
2001


22
Roller Blade
3.1
1986


23
Golden Ninja Warrior
3.1
1986


24
Bloodlust
3.1
1992


25
Lethal Ninja
3.1
1992


26
L.A. Task Force
3.1
1994


27
Dinosaur Valley Girls
3.1
1996


28
Vice Girls
3.1
1997


29
Airborne
3.1
1998


30
Naked Lies
3.1
1998


31
Ninja vs. Ninja
3.1
1987


32
The Lost Idol
3.1
1989


33
Militia
3.1
2000


34
Aaj Ke Shahenshah
3.1
1990


35
Blind Target
3.1
2000


36
Naked Assassins
3.1
1996


37
Mindstorm
3.1
2001


38
Sev tevi tak
3.1
1930


39
Amazons and Gladiators
3.1
2001


40
Billy Boy
3.1
1978


41
Operation Las Vegas
3.1
1990


42
Ang utol kong hoodlum
3.1
1991


43
Post Impact
3.1
2004


44
The Unknown
3.1
2005


45
Dinocroc
3.1
2004


46
Janasheen
3.1
2003


47
Rottweiler
3.1
2004


48
Triads, Yardies & Onion Bhajees!
3.1
2003


49
Vajram
3.1
2004


50
Breathe Safely
3.1
1998


1
Far Cry
3.1
2008


2
Sullan
3.1
2004


3
Southern Justice
3.1
2006


4
Horrors of War
3.1
2006


5
Pterodactyl
3.1
2005


6
Moscow Zero
3.1
2006


7
Sarisin tehlike
3.1
1980


36
Sins of the Realm
3.3
2003


37
The Purifiers
3.3
2004


38
Juaari
3.3
1994


39
Daulat Ke Dushman
3.3
1983


40
The Witches Hammer
3.3
2006


41
Action Jackson
3.3
2014


India
42
Clementine
3.3
2004


43
Royal Kill
3.3
2009


44
Aaj Ka Andha Kanoon
3.3
2003


45
Segurança Nacional
3.3
2010


46
38. expansiva
3.3
1996


47
Fight Club: Members Only
3.3
2006


48
Pyare Mohan
3.3
2006


49
Lisensyadong kamao
3.3
2005


50
Displaced
3.3
2006


1
Last Hour
3.3
2008


2
Ouzo, Stoff und Taschentücher
3.3
2006


3
Gajendra
3.3
2004


4
TKO
3.3
2007


5
Tirupathi
3.3
2006


6
Sukran
3.3
2005


7
The Vanguard
3.3
2008


8
Resiklo
3.3
2007


9
Hashtpa
3.3
2005


10
The Way of War
3.3
2009


11
Blood Bound
3.3
2007


12
Tales of an Ancient Empire
3.3
2010


13
Bricktown
3.3
2008


14
The Korean
3.3
2008


15
El-shayatin: El-Awdah
3.3
2007


16
Stagger
3.3
2009


17
Unarmed But Dangerous
3.3
2009


18
Bullet
3.3
2008


19
Shardhanjali
3.3
1994


20
Mottomo abunai deka
3.3
1989


21
Cold Fusion


...

30
Tied in Blood
3.7
2012


31
Éxito por intercambio
3.7
2003


32
The Beautiful and Damned
3.7
2010


33
Japan Japan
3.7
2007


34
Mohakemeh
3.7
2007


35
Lokanathan I.A.S
3.7
2005


36
Saddam
3.7
2006


37
Main Aurr Mrs Khanna
3.7
2009


38
Murder.com
3.7
2008


39
Yumejuya: Kaizokuban
3.7
2007


40
Hark sei nei
3.7
2008


41
Kalidasu
3.7
2008


42
The Legends of Nethiah
3.7
2012


43
The Scar Crow
3.7
2009


44
Ornaments
3.7
2008


45
Finale
3.7
2009


46
Fast Lane
3.7
2010


47
Know Thy Enemy
3.7
2009


48
Do Knot Disturb
3.7
2009


49
The Honey Killer
3.7
2011


50
Bishh
3.7
2009


1
Jigar
3.7
1998


2
Project: Threshold
3.7
2009


3
I Love You 2
3.7
2009


4
Germ
3.7
2013


USA
5
L'ultimo ultras
3.7
2009


6
Di Di Hollywood
3.7
2010


7
El calambre
3.7
2009


8
Five Hours South
3.7
2012


9
Wrecked
3.7
2009


10
Axed
3.7
2012


11
Bunkerember
3.7
2010


12
Break My Fall
3.7
2011


13
Ada... A Way of Life
3.7
2010


14
Dam999
3.7
2011


15
Pramani
3.7
2010


16
Plus Two
3.7
2010



41
Thomas and the Magic Railroad
3.8
2000


42
The Deathhead Virgin
3.8
1974


43
La liberté en croupe
3.8
1970


44
Les gardiennes du pénitencier
3.8
1979


45
Helga, la louve de Stilberg
3.8
1978


46
Boricua's Bond
3.8
2000


47
Masho no natsu - 'Yotsuya kaidan' yori
3.8
1981


48
Shikake-nin Baian
3.8
1981


49
O dromos me ta kokkina fota
3.8
1963


50
Irthes arga
3.8
1962


1
Mela
3.8
2000


2
Extreme Vengeance
3.8
1990


3
Against All Hope
3.8
1982


4
The Divorce
3.8
1903


5
Blood Circle
3.8
1994


6
Janaina - A Virgem Proibida
3.8
1972


7
Fernes Land Pa-Isch
3.8
2000


8
Basuras humanas
3.8
1972


9
Losing Grace
3.8
2001


10
One, Two, Two: 122, rue de Provence
3.8
1978


11
Pilotos de combate
3.8
1973


12
Den gnorisa mitera
3.8
1962


13
Dhaai Akshar Prem Ke
3.8
2000


14
Doxobus
3.8
1987


15
Emotional Backgammon
3.8
2003


16
Katrakylisma... sto vourko
3.8
1962


17
Ajay
3.8
1996


18
Prem Shakti
3.8
1994


19
Raja Ki Ayegi Baraat
3.8
1997


20
El diablo desaparece
3.8
19

6
L'amour des hommes
3.8
2017


France
7
Charlotte
3.8
2017


Australia
8
Hoe kamelen leeuwen worden
3.8
2017


Belgium
9
Salar-e mardan
3.8
1968


10
Octavio Is Dead!
3.8
2018


11
Anbe Anbe
3.8
2003


12
The Last Verse
3.8
2017


Taiwan
13
X Ma'anah Alamt Khata
3.8
1980


14
Speed Kills
3.8
2018


15
Mon tissu préféré
3.8
2018


16
Hadi Badi
3.8
1984


17
M.I.A. A Greater Evil
3.8
2018


18
Birds of a Feather 2
3.8
2018


19
The Thinning: New World Order
3.8
2018


20
Akhlaa El-Abid
3.8
2017


Egypt
21
Doglibjeon-ya
3.8
1948


22
Deckname Jenny
3.8
2017


Germany
23
Long Time, Love
3.8
1973


24
Hamlet
3.9
1911


25
Usinatlo
3.9
1930


26
Tukkipojan morsian
3.9
1931


27
Ne 45000
3.9
1933


28
Racetrack
3.9
1933


29
Marihuana
3.9
1936


30
On Probation
3.9
1935


31
Éjjeli zene
3.9
1943


32
The Brute Man
3.9
1946


33
Brindis a Manolete
3.9
1948


34
The Devil's Sleep
3.9
1949


35
María Morena
3.9
1951


36
Säg det med blommor
3.9
1952


37
Los ases buscan la paz
3.9
1955


38
Nie

27
Roadside
3.9
2013


USA
28
King Khan
3.9
2011


29
Hollow
3.9
2011


30
Hum Tum Shabana
3.9
2011


31
Taking Stock
3.9
2015


UK
32
Miley - Naa Miley - Hum
3.9
2011


33
Held for Ransom
3.9
1976


34
The Innocent Sinners
3.9
2012


35
Kalevala - uusi aika
3.9
2013


Finland
36
Maìn - La casa della felicità
3.9
2012


37
To Hell with a Bullet
3.9
2013


USA
38
Payyans
3.9
2011


39
Sine-chak
3.9
1977


40
Tympanum
3.9
2012


41
Aal
3.9
2010


42
Free the Nipple
3.9
2014


USA
43
Hope for Love
3.9
2013


USA
44
The Darkening
3.9
2012


45
Justice Squad
3.9
2012


46
Believe Again
3.9
2013


USA
47
Het bombardement
3.9
2012


48
The Disappointments Room
3.9
2016


USA
49
Criticsized
3.9
2016


USA
50
Dead Within
3.9
2014


USA
1
Kaderim
3.9
1987


2
Sabbubah
3.9
2012


3
El lenguaje del tiempo
3.9
2011


4
Ambushed
3.9
2013


USA
5
Bez doteku
3.9
2013


Czech Republic
6
Muzhskaya zhenskaya igra
3.9
2011


7
Annunciation to a Third Milennium Citizen
3.9
2013


Iran
8
Scene Onnu Nammude 

ConnectionError: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0096770/?ref_=adv_li_tt (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002CF09D7A978>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [27]:
IMDBmovie = {
    'title': movie_title,
    'date': movie_date,
    'country': movie_country,
    'genre': movie_genre,
    'rate': movie_rate,
}

In [29]:
pd_IMDB = pd.DataFrame.from_dict(IMDBmovie, orient ='index').transpose()
pd_IMDB.to_csv('pd_IMDB.csv')

for line in locals()['In']:
    print(line)

#或者， 直接的
history